In [56]:
import pandas as pd
import numpy as np
import sqlite3

In [57]:
titanic_df = pd.read_csv(r"/Users/abhiramsingireddy/.cache/kagglehub/datasets/brendan45774/test-file/versions/6/tested.csv")

In [58]:
titanic_df = titanic_df.set_index("PassengerId")

In [59]:
titanic_df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [60]:
# creating the db... connecting to a db that doesn't exist creates and connects
conn = sqlite3.connect('titanic.db')

In [61]:
# migrates dataframe to the db, Replaces if exists already
titanic_df.to_sql('tbl_titanic', con=conn, if_exists='replace')

418

In [62]:
test_sql = """
    SELECT
        PassengerID, 
        Survived, 
        Pclass, 
        Name, 
        Age, 
        Fare
FROM tbl_titanic
WHERE
    Pclass in 1
    AND AGE < 50
ORDER BY Age desc
"""

In [73]:
test_sql = """
    SELECT
        Pclass, 
        Survived,
        Avg(Age)
FROM tbl_titanic
WHERE
    Pclass in (1,2)
GROUP BY 
    Pclass,
    Survived
"""

In [74]:
sql_df = pd.read_sql(test_sql, con=conn)

sql_df

,Pclass,Survived,Avg(Age)
0,1,0,40.520000
1,1,1,41.333333
2,2,0,30.940678
3,2,1,24.376552


In [88]:
sql_string = """
SELECT
    Pclass,
    CASE
        WHEN Age < 18 THEN 'child'
        ELSE 'adult'
    END AS age_group,
    CAST(Avg(Age) as integer) AS average_age
FROM tbl_titanic
WHERE Embarked = 'C'
GROUP BY
    Pclass,
    CASE
        WHEN Age < 18 THEN 'child'
        ELSE 'adult'
    END;
"""

In [86]:
sql_df2 = pd.read_sql(sql_string, con=conn)

sql_df2

,Pclass,age_group,average_age
0,1,adult,41
1,1,child,9
2,2,adult,28
3,2,child,1
4,3,adult,26
5,3,child,9


In [89]:
!pip install sqlalchemy


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [90]:
from sqlalchemy import create_engine, text

In [96]:
engine = create_engine('sqlite:///titanic.db')

In [97]:
with engine.connect() as connection:
    sql = text("SELECT * FROM tbl_titanic WHERE PassengerId = 894")
    titanic_sa_df = pd.read_sql(sql, connection)

In [98]:
titanic_sa_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
